# Monte Carlo Tree Search

## Imports

These are all the imports needed to solve this task:

In [2]:
from heapq import heappop, heappush
from pocket_cube.cube import Cube
from pocket_cube.cube import Move
from tests import case1, case2, case3, case4
import numpy as np
import matplotlib.pyplot as plt
from random import choice

%matplotlib notebook

## Test cases

These are the test cases that will be used to run and compare the algorithms:

In [3]:
cube1 = Cube(case1)
cube2 = Cube(case2)
cube3 = Cube(case3)
cube4 = Cube(case4)

For this problem, we will define some constants:

In [4]:
# Constants

N = 'N'
Q = 'Q'
P = 'parent'
A = 'actions'

This is the first heuristic, the one that I used on the previous task. It is admissible. The return value is 3 minus the heuristic, because the maximum value this heuristic can give is 3, so in order to make it a "reward", not a "distance", we do this operation.

In [5]:
def heuristic_score(cube):
        score = 0
        for i in range(len(cube.state)):
            if cube.goal_state[i] != cube.state[i]:
                score += 1
    
        return 3 - score / 8

This is the second heuristic that uses the Manhattan Distance.
This heuristic is not admissible because the aproximated value will be bigger than the real value. The return value is 24 minus the result of the heuristic, because there are 8 corners, and the maximum distance a corner can be from his correct position is 3, so in order to make it a "reward", we do this operation.

In [6]:
def Manhattan_calc(coord1, coord2):
        return abs(coord1[0] - coord2[0]) + \
               abs(coord1[1] - coord2[1]) + \
               abs(coord1[2] - coord2[2])

def manhattan_distance_not_admissible(cube):
        
        dict_cube = {}
        score_manh = 0.0
        for coord, (cublets_coord, _) in cube.cublets.items():
            x, y, z = cublets_coord
            color_x = cube.state[x]
            color_y = cube.state[y]
            color_z = cube.state[z]
            dict_cube[coord] = ((x, y, z), (color_x, color_y, color_z))
    
        for coord, (_, goal_colors) in cube.cublets.items():
            _, cublets_colors = dict_cube[coord]
            if goal_colors != cublets_colors:
                for cublets_coord in dict_cube.keys():
                    if set(dict_cube[cublets_coord][1]) == set(goal_colors):
                        score_manh += Manhattan_calc(coord, cublets_coord)

        return 24 - score_manh

In [7]:
def is_solved(cube):
    comparator = cube.goal_state == cube.state
    return comparator.all()

In [8]:
def get_neighbours(cube):
    return [Cube.hash_state(Cube.move_state(cube.clone_state(), move)) for move in range(6)]

In [9]:
def init_node(parent=None):
    return {N: 0, Q: 0, P: parent, A: {}}

In [10]:
from math import sqrt, log
import numpy as np

def select_action(node, c):
    max_score = -1
    action = None
    
    for a, n in node[A].items():
        score = n[Q] / n[N] + c * sqrt(2 * log(node[N]) / n[N])
        if max_score < score:
            max_score = score
            action = a
    
    return action

In [11]:
def mcts(cube, budget, tree, heuristic, C):
    c = C
    count = 0
    solved = False
    if not tree:
        tree = init_node()
    
    for b in range(budget):
        
        probe_cube = cube.clone()
        node = tree
        solution = [] 
        while(not is_solved(probe_cube) and 
            all(move in node[A] for move in [x for x in range(6)])):
            selected_move = select_action(node, c)
            probe_cube.state = Cube.move_state(probe_cube.state, selected_move)
            solution.append(probe_cube.hash())
            node = node[A][selected_move]
        
        if is_solved(probe_cube):
            print("solved cube!")
            print(f"Total number of nodes discovered: {count}")
            print(f"Length of the solution is: {str(len(solution))}")
            print("Solution is:")
            print(solution)
            solved = True
            break
        else:
            new_move = choice(list(filter(lambda a: a not in node[A], [x for x in range(6)])))
                       
            probe_cube.state = Cube.move_state(probe_cube.state, new_move)
            node = init_node(node)
            count += 1
            node[P][A][new_move] = node
            
        tries = 0
        max_reward = 0
        while tries < 14:
            # I'm searching for the max reward possible on this route
            if max_reward < heuristic(probe_cube):
                max_reward = heuristic(probe_cube)
            
            if is_solved(probe_cube):
                break
                
            probe_cube.state = Cube.move_state(probe_cube.state, choice([x for x in range(6)]))
            tries += 1
            
        crt_node = node
        while crt_node:
            crt_node[N] += 1
            crt_node[Q] += max_reward
            crt_node = crt_node[P]
    
    
    if not solved:
        print('NO SOLUTION FOUND')

In [12]:
def run_case1(budget, H, C):
    mcts(cube1, budget, None, H, C)

def run_case2(budget, H, C):
    mcts(cube2, budget, None, H, C)
    
def run_case3(budget, H, C):
    mcts(cube3, budget, None, H, C)
    
def run_case4(budget, H, C):
    mcts(cube4, budget, None, H, C)

## We will test the algorithm for the first heuristic with budget 1000, and c = 0,1

In [14]:
%%time
for i in range(20):
    run_case1(1000, heuristic_score, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 154
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 102
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 76
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube

In [15]:
%%time
for i in range(20):
    run_case2(1000, heuristic_score, 0.1)

solved cube!
Total number of nodes discovered: 416
Length of the solution is: 9
Solution is:
['515104033204322240351415', '512204513203320440351154', '502541054213320440331152', '055211154213330340402452', '005015112243330341442552', '000555212243340441113352', '010151522233340444123055', '001111222233330044445555', '000011112222333344445555']
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 147
Length of the solution is: 7
Solution is:
['105043415224352340032115', '001513215224332040445315', '002113245220331540443551', '052121434200331540413255', '051521214243330040415352', '001111222233330044445555', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 162
Length of the solution is: 7
Solution is:
['105043415224352340032115', '001513215224332040445315'

In [16]:
%%time
for i in range(20):
    run_case3(1000, heuristic_score, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 10.5 s
Wall time: 10.4 s


In [17]:
%%time
for i in range(20):
    run_case4(1000, heuristic_score, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 11.3 s
Wall time: 11.2 s


## We will test the algorithm for the first heuristic with budget 5000, and c = 0,1

In [18]:
%%time
for i in range(20):
    run_case1(5000, heuristic_score, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 176
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 46.9 s
Wall time: 46.9 s


In [19]:
%%time
for i in range(20):
    run_case2(5000, heuristic_score, 0.1)

NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 1109
Length of the solution is: 15
Solution is:
['112344553213350440001252', '110444233255351340005122', '100043422215351345035124', '001013522215331045445324', '001013102252331545442543', '051311034252331540402542', '051511134203335240404225', '055211154213330340402452', '005015112243330341442552', '500045412243320541331152', '005015112243330341442552', '000555212243340441113352', '010151522233340444123055', '001111222233330044445555', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 314
Length of the solution is: 7
Solution is:
['105043415224352340032115', '001513215224332040445315', '002113245220331540443551', '052121434200331540413255', '051521214243330040415352', '001111222233330044445555', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FO

In [20]:
%%time
for i in range(20):
    run_case3(5000, heuristic_score, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 4001
Length of the solution is: 21
Solution is:
['321514013205353240254104', '311401145205353242254003', '321514013205353240254104', '301541104215353240234205', '053121404215333240415205', '054021154232333140412550', '053121404215333240415205', '053221314240331540410552', '003111232250331544440552', '001511312223335044445025', '105041412223355044033125', '001511312223335044445025', '041411135203335042425025', '045011145213330342422550', '055011412223330344402155', '500511212223300444143355', '055011412223330344402155', '050311502241332344405251', '003541002241352244335151', '000041412222353544331155', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 1664
Length of the solution is: 13
Solution is:
['313201145214350542250430', '343010415224350541220135', '340510305241352441223051', '300131002

In [21]:
%%time
for i in range(20):
    run_case4(5000, heuristic_score, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 1min 6s
Wall time: 1min 6s


## We will test the algorithm for the first heuristic with budget 10000, and c = 0,1

In [22]:
%%time
for i in range(20):
    run_case1(10000, heuristic_score, 0.1)

solved cube!
Total number of nodes discovered: 105
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 6182
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 1min 35s
Wall time: 1min 35s


In [23]:
%%time
for i in range(20):
    run_case2(10000, heuristic_score, 0.1)

solved cube!
Total number of nodes discovered: 101
Length of the solution is: 7
Solution is:
['105043415224352340032115', '001513215224332040445315', '002113245220331540443551', '052121434200331540413255', '051521214243330040415352', '001111222233330044445555', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 253
Length of the solution is: 7
Solution is:
['105043415224352340032115', '001513215224332040445315', '002113245220331540443551', '052121434200331540413255', '051521214243330040415352', '001111222233330044445555', '000011112222333344445555']
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 114
Length of the solution is: 7
Solution is:
['105043415224352340032115', '001513215224332040445315', '00211324522033154

In [24]:
%%time
for i in range(20):
    run_case3(10000, heuristic_score, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 2min 10s
Wall time: 2min 10s


In [25]:
%%time
for i in range(20):
    run_case4(10000, heuristic_score, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 2min 24s
Wall time: 2min 24s


## We will test the algorithm for the first heuristic with budget 20000, and c = 0,1

In [26]:
%%time
for i in range(20):
    run_case1(20000, heuristic_score, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 10967
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 161
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 3min 18s
Wall time: 3min 18s


In [27]:
%%time
for i in range(20):
    run_case2(20000, heuristic_score, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 4min 51s
Wall time: 4min 51s


In [28]:
%%time
for i in range(20):
    run_case3(20000, heuristic_score, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 4min 51s
Wall time: 4min 51s


In [29]:
%%time
for i in range(20):
    run_case4(20000, heuristic_score, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 5min 20s
Wall time: 5min 21s


## We will test the algorithm for the first heuristic with budget 1000, and c = 0,5

In [30]:
%%time
for i in range(20):
    run_case1(1000, heuristic_score, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 879
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 880
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 377
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 596
Length of the

In [31]:
%%time
for i in range(20):
    run_case2(1000, heuristic_score, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 7.22 s
Wall time: 7.22 s


In [32]:
%%time
for i in range(20):
    run_case3(1000, heuristic_score, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 7.11 s
Wall time: 7.1 s


In [33]:
%%time
for i in range(20):
    run_case4(1000, heuristic_score, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 7.12 s
Wall time: 7.12 s


## We will test the algorithm for the first heuristic with budget 5000, and c = 0,5

In [34]:
%%time
for i in range(20):
    run_case1(5000, heuristic_score, 0.5)

solved cube!
Total number of nodes discovered: 4133
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 2801
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 1427
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 819
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 3351
Length of

In [35]:
%%time
for i in range(20):
    run_case2(5000, heuristic_score, 0.5)

solved cube!
Total number of nodes discovered: 2085
Length of the solution is: 7
Solution is:
['105043415224352340032115', '001513215224332040445315', '002113245220331540443551', '052121434200331540413255', '051521214243330040415352', '001111222233330044445555', '000011112222333344445555']
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 3186
Length of the solution is: 7
Solution is:
['105043415224352340032115', '001513215224332040445315', '002113245220331540443551', '052121434200331540413255', '051521214243330040415352', '001111222233330044445555', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 2015
Length of the solution is: 7
Solution is:
['105043415224352340032115', '001513215224332040445315', '002113245220331540443551', '052121434200331540413255', '051521214243330040415352', '001111222233330044445555', '000011112222333344445555']
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 4492
Len

In [36]:
%%time
for i in range(20):
    run_case3(5000, heuristic_score, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 42.6 s
Wall time: 42.6 s


In [37]:
%%time
for i in range(20):
    run_case4(5000, heuristic_score, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 46.8 s
Wall time: 46.9 s


## We will test the algorithm for the first heuristic with budget 10000, and c = 0,5

In [38]:
%%time
for i in range(20):
    run_case1(10000, heuristic_score, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 5087
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 1872
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 1578
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 836
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
solved cube!
Total number

In [39]:
%%time
for i in range(20):
    run_case2(10000, heuristic_score, 0.5)

NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 3475
Length of the solution is: 7
Solution is:
['105043415224352340032115', '001513215224332040445315', '002113245220331540443551', '052121434200331540413255', '051521214243330040415352', '001111222233330044445555', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 2010
Length of the solution is: 7
Solution is:
['105043415224352340032115', '001513215224332040445315', '002113245220331540443551', '052121434200331540413255', '051521214243330040415352', '001111222233330044445555', '000011112222333344445555']
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 3229
Length of the solution is: 7
Solution is:
['105043415224352340032115', '001513215224332040445315', '002113245220331540443551', '052121434200331540413255', '051521214243330040415352', '001111222233330044445555', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 3400
Length of the solution is: 7
Solution i

In [40]:
%%time
for i in range(20):
    run_case3(10000, heuristic_score, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 1min 36s
Wall time: 1min 36s


In [41]:
%%time
for i in range(20):
    run_case4(10000, heuristic_score, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 1min 52s
Wall time: 1min 55s


## We will test the algorithm for the first heuristic with budget 20000, and c = 0,5

In [42]:
%%time
for i in range(20):
    run_case1(20000, heuristic_score, 0.5)

solved cube!
Total number of nodes discovered: 5869
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 1566
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 5425
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 2288
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 1179
Length o

In [43]:
%%time
for i in range(20):
    run_case2(20000, heuristic_score, 0.5)

NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 2126
Length of the solution is: 7
Solution is:
['105043415224352340032115', '001513215224332040445315', '002113245220331540443551', '052121434200331540413255', '051521214243330040415352', '001111222233330044445555', '000011112222333344445555']
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 2683
Length of the solution is: 7
Solution is:
['105043415224352340032115', '001513215224332040445315', '002113245220331540443551', '052121434200331540413255', '051521214243330040415352', '001111222233330044445555', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 3723
Length of the solution is: 7
Solution is:
['105043415224352340032115', '001513215224332040445315', '002113245220331540443551', '052121434200331540413255', '051521214243330040415352', '001111222233330044445555', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 1538
Length of the solution is: 7
Solution i

In [44]:
%%time
for i in range(20):
    run_case3(20000, heuristic_score, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 11588
Length of the solution is: 11
Solution is:
['313201145214350542250430', '343010415224350541220135', '340510305241352441223051', '300131002211352444253455', '013041302211352244305455', '100341502211302344435255', '105041112223300344432555', '001551212223330444410355', '041111525233330442420055', '040411115252333342425050', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 3min 40s
Wall time: 3min 47s


In [45]:
%%time
for i in range(20):
    run_case4(20000, heuristic_score, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 4min 4s
Wall time: 4min 11s


## We will test the algorithm with the second heuristic with budget 1000, and c = 0,1

In [46]:
%%time
for i in range(20):
    run_case1(1000, manhattan_distance_not_admissible, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 73
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 127
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
CPU times: total: 18.2 s
Wall time: 18.2 s


In [47]:
%%time
for i in range(20):
    run_case2(1000, manhattan_distance_not_admissible, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 21.1 s
Wall time: 21.1 s


In [48]:
%%time
for i in range(20):
    run_case3(1000, manhattan_distance_not_admissible, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 21.8 s
Wall time: 21.8 s


In [49]:
%%time
for i in range(20):
    run_case4(1000, manhattan_distance_not_admissible, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 23.7 s
Wall time: 24.2 s


## We will test the algorithm with the second heuristic with budget 5000, and c = 0,1

In [50]:
%%time
for i in range(20):
    run_case1(5000, manhattan_distance_not_admissible, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 36
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 2min 5s
Wall time: 2min 6s


In [51]:
%%time
for i in range(20):
    run_case2(5000, manhattan_distance_not_admissible, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 2min 10s
Wall time: 2min 11s


In [52]:
%%time
for i in range(20):
    run_case3(5000, manhattan_distance_not_admissible, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 2min 27s
Wall time: 2min 33s


In [53]:
%%time
for i in range(20):
    run_case4(5000, manhattan_distance_not_admissible, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 2min 23s
Wall time: 2min 27s


## We will test the algorithm with the second heuristic with budget 10000, and c = 0,1

In [54]:
%%time
for i in range(20):
    run_case1(10000, manhattan_distance_not_admissible, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 63
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 92
Length of the solution is: 7
Solution is:
['254211342200355044101533', '255011422234350044103135', '502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 4min 47s
Wall time: 4min 49s


In [55]:
%%time
for i in range(20):
    run_case2(10000, manhattan_distance_not_admissible, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 5min 4s
Wall time: 5min 5s


In [56]:
%%time
for i in range(20):
    run_case3(10000, manhattan_distance_not_admissible, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 5min 16s
Wall time: 5min 16s


In [57]:
%%time
for i in range(20):
    run_case4(10000, manhattan_distance_not_admissible, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 5min 16s
Wall time: 5min 16s


## We will test the algorithm with the second heuristic with budget 20000, and c = 0,1

In [58]:
%%time
for i in range(20):
    run_case1(20000, manhattan_distance_not_admissible, 0.1)

solved cube!
Total number of nodes discovered: 107
Length of the solution is: 9
Solution is:
['502511322234300144145035', '502531214244300140155332', '502511322234300144145035', '500111252232303444143550', '540415120252303443123051', '530252111202303445103454', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 63
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
NO SOLUTION FOUND
CPU times: total: 10min 16s
Wall time: 10min 17s


In [59]:
%%time
for i in range(20):
    run_case2(20000, manhattan_distance_not_admissible, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 10min 54s
Wall time: 10min 55s


In [22]:
%%time
for i in range(20):
    run_case3(20000, manhattan_distance_not_admissible, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 11min 1s
Wall time: 11min 1s


In [23]:
%%time
for i in range(20):
    run_case4(20000, manhattan_distance_not_admissible, 0.1)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 10min 55s
Wall time: 10min 55s


## We will test the algorithm with the second heuristic with budget 1000, and c = 0.5

In [13]:
%%time
for i in range(20):
    run_case1(1000, manhattan_distance_not_admissible, 0.5)

solved cube!
Total number of nodes discovered: 856
Length of the solution is: 7
Solution is:
['502511322234300144145035', '500111252232303444143550', '051011452232333544402150', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 72
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 16.5 s
Wall time: 16.5 s


In [15]:
%%time
for i in range(20):
    run_case2(1000, manhattan_distance_not_admissible, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 18.4 s
Wall time: 18.3 s


In [16]:
%%time
for i in range(20):
    run_case3(1000, manhattan_distance_not_admissible, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 19.2 s
Wall time: 19.2 s


In [17]:
%%time
for i in range(20):
    run_case4(1000, manhattan_distance_not_admissible, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 19 s
Wall time: 19 s


## We will test the algorithm with the second heuristic with budget 5000, and c = 0.5

In [18]:
%%time
for i in range(20):
    run_case1(5000, manhattan_distance_not_admissible, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 99
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 1min 33s
Wall time: 1min 33s


In [19]:
%%time
for i in range(20):
    run_case2(5000, manhattan_distance_not_admissible, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 1min 40s
Wall time: 1min 40s


In [20]:
%%time
for i in range(20):
    run_case3(5000, manhattan_distance_not_admissible, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 1min 45s
Wall time: 1min 45s


In [21]:
%%time
for i in range(20):
    run_case4(5000, manhattan_distance_not_admissible, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 1min 45s
Wall time: 1min 44s


## We will test the algorithm with the second heuristic with budget 10000, and c = 0.5

In [24]:
%%time
for i in range(20):
    run_case1(10000, manhattan_distance_not_admissible, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 129
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 122
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
solved cube!
Total number of nodes discovered: 180
Length of the solution is: 7
Solution is:
['502511322234300144145035', '500111252232303444143550', '051011452232333544402150', '500111252232303444143550', '550021514242303440113352', '050511114

In [25]:
%%time
for i in range(20):
    run_case2(10000, manhattan_distance_not_admissible, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 3min 38s
Wall time: 3min 38s


In [26]:
%%time
for i in range(20):
    run_case3(10000, manhattan_distance_not_admissible, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 3min 38s
Wall time: 3min 38s


In [27]:
%%time
for i in range(20):
    run_case4(10000, manhattan_distance_not_admissible, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 3min 43s
Wall time: 3min 43s


## We will test the algorithm with the second heuristic with budget 20000, and c = 0.5

In [28]:
%%time
for i in range(20):
    run_case1(20000, manhattan_distance_not_admissible, 0.5)

solved cube!
Total number of nodes discovered: 72
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 7min
Wall time: 7min 1s


In [29]:
%%time
for i in range(20):
    run_case2(20000, manhattan_distance_not_admissible, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 7min 40s
Wall time: 7min 40s


In [30]:
%%time
for i in range(20):
    run_case3(20000, manhattan_distance_not_admissible, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 7min 49s
Wall time: 7min 49s


In [31]:
%%time
for i in range(20):
    run_case4(20000, manhattan_distance_not_admissible, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 8min 34s
Wall time: 8min 34s
